# DATA ENGINEERING ETL PROCESS 

The objective of this exercise is to run an **ETL** process:<br>
*   Run the ETL process
*   Extract bank and market cap data from the JSON file `bank_market_cap.json`
*   Transform the market cap currency using the exchange rate data
*   Load the transformed data into a seperate CSV

In [31]:
import glob
import pandas as pd
from datetime import datetime
import urllib.request

### DOWNLOAD THE FILES

In [32]:
url_1 = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json"

url_2 = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json"

url_3 = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv"

download_1 = urllib.request.urlretrieve(url_1, "ETL_BANK_MARKET_CAP_1.json")
download_2 = urllib.request.urlretrieve(url_2, "ETL_BANK_MARKET_CAP_2.json")
download_3 = urllib.request.urlretrieve(url_3, "ETL_EXCHANGE_RATES.csv")

## **EXTRACT**

### FUNCTION TO EXTRACT JSON DATA

In [33]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

### FUNCTION TO EXTRACT THE DATA AND SAVE IT IN A DATA FRAME

In [34]:
columns=['Name','Market Cap (US$ Billion)']
def extract():
    extracted_data = pd.DataFrame(columns=columns)

    for json in glob.glob("ETL_BANK_MARKET_CAP_1.json"):
        extracted_data = extracted_data.append(extract_from_json(json), ignore_index=True)
    return extracted_data



## **TRANSFORM**

### OBTAINING THE **GBP** RATE TO USE IT LATER

In [35]:
exchange_rates = pd.read_csv("ETL_EXCHANGE_RATES.csv", header=None)
exchange_rates.columns = ["CURRENCY", "RATES"]
exchange_rates = exchange_rates.iloc[1:,0:2] 


In [48]:
exchange_rate = exchange_rates.loc[exchange_rates["CURRENCY"]=="GBP"]
exchange_rate = exchange_rate.iloc[0,1]
exchange_rate = float(exchange_rate)
print("exchange_ratae is:", type(exchange_rate),"and equals to", exchange_rate)

exchange_ratae is: <class 'float'> and equals to 0.7323984208000001


### TRANSFORMING THE DATA

Using <code>exchange_rate</code> and the "ETL_EXCHANGE_RATES.csv" file find the exchange rate of USD to GBP. Write a transform function that

1.  Changes the `Market Cap (US$ Billion)` column from USD to GBP
2.  Rounds the Market Cap (US$ Billion)\` column to 3 decimal places
3.  Rename `Market Cap (US$ Billion)` to `Market Cap (GBP$ Billion)`

In [68]:
data_extract = extract()
cap_us=data_extract[["Market Cap (US$ Billion)"]]

C:\Users\Miguel Lizarraga\AppData\Local\Temp\ipykernel_5932\1881791872.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_json(json), ignore_index=True)


In [ ]:
def transform(data):
    data["Market Cap (US$ Billion)"] = data["Market Cap (US$ Billion)"].astype(float)
    data["Market Cap (US$ Billion)"] = round(data["Market Cap (US$ Billion)"]*exchange_rate,3)
    data["Market Cap (GPB$ Billion)"] = data["Market Cap (US$ Billion)"]
    df_new = pd.DataFrame()
    df_new["Name"] = data.iloc[0:,0:1]
    cap_us = data_extract[["Market Cap (US$ Billion)"]]
    df_new["Market Cap (GBP$ Billion)"] = cap_us
    return df_new

transform(data_extract)

## **LOAD**

### LOAD THE DATA INTO A CSV FILE

In [39]:
target_file = "bank_market_cap_gbp.csv"
def load(target_file, data_to_load):
    data_to_load.to_csv(target_file, index=False)

## **LOGG FUNCTION**

### LOGGING FUNCTION TO REGISTER THE TIMESTAMP INTO A TXT FILE

In [40]:
def log(description_process):
    timestamp_format = "%Y-%h-%d-%H:%M:%S"
    now = datetime.now()
    time_stamp = now.strftime(timestamp_format)
    with open ("registers.txt", "a") as logg:
        logg.write(time_stamp +" ---> " + description_process + "\n")

## **RUNNING THE EXTRACT, TRANSFORM, LOAD PROCESS**

### EXTRACT PHASE

In [41]:
log("ELT Job Started")
log("Extract phase Started")

In [42]:
extract()
print(extract().head())

                                      Name Market Cap (US$ Billion)
0                           JPMorgan Chase                  390.934
1  Industrial and Commercial Bank of China                  345.214
2                          Bank of America                  325.331
3                              Wells Fargo                  308.013
4                  China Construction Bank                  257.399


C:\Users\Miguel Lizarraga\AppData\Local\Temp\ipykernel_5932\1881791872.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_json(json), ignore_index=True)
C:\Users\Miguel Lizarraga\AppData\Local\Temp\ipykernel_5932\1881791872.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_json(json), ignore_index=True)


In [43]:
log("Extract phase Ended")

### TRANSFORM PHASE

In [44]:
log("Transfor phase Started")

In [45]:
data_to_transform = extract()
print("ORIGINAL DATA: \n\n", data_to_transform.head()) #ORIGINAL DATA

ORIGINAL DATA: 

                                       Name Market Cap (US$ Billion)
0                           JPMorgan Chase                  390.934
1  Industrial and Commercial Bank of China                  345.214
2                          Bank of America                  325.331
3                              Wells Fargo                  308.013
4                  China Construction Bank                  257.399


C:\Users\Miguel Lizarraga\AppData\Local\Temp\ipykernel_5932\1881791872.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_json(json), ignore_index=True)


In [62]:
print("TRANSFORMED DATA: \n\n",transform(data_to_transform))


TRANSFORMED DATA: 

                                        Name  Market Cap (GBP$ Billion)
0                            JPMorgan Chase                    286.319
1   Industrial and Commercial Bank of China                    252.834
2                           Bank of America                    238.272
3                               Wells Fargo                    225.588
4                   China Construction Bank                    188.519
..                                      ...                        ...
65                             Ping An Bank                     27.826
66                       Standard Chartered                     27.332
67                     United Overseas Bank                     25.728
68                                QNB Group                     24.579
69                              Bank Rakyat                     24.228

[70 rows x 2 columns]


### LOAD PHASE

In [64]:
log("Load phase Started")

In [65]:
transformed_data = transform(data_to_transform)
target_file = "bank_market_cap_gbp.csv"
load(target_file,transformed_data)

In [66]:
log("Load phase Ended")